In [5]:
import requests
import pandas as pd
from utils.helper import Proxies

proxies = Proxies(annoymous=False, from_env=True)
# useragent = proxies.get_proxy_useragent()
proxy_ip = proxies.get_proxy_ip()

proxy = {
    "http": proxy_ip,
    "https": proxy_ip
}
# headers = {"Connection" : "close",  "User-Agent" : useragent}  
# print("User Agent:", useragent)
# print("Proxy ip:", proxy_ip)


# Google News

In [ ]:
import requests
import json
import traceback
import time
import bs4 as bs
import pandas as pd
import urllib.parse
from datetime import datetime, timedelta, date
from IPython.display import display

class GoogleRSSFeedScraper():
    def __init__(self, proxy=None):
        self.proxy = proxy
        self.state = {}
        self.base_url = "https://news.google.com/rss/search?"
    
    def scrape(self, search_query, start_time=datetime.now().date(), end_time = datetime.now().date(), scrape_period = 7, add_url_parameters={}, save_state=False, save_result=False, save_folder="./data", preview_result=True, sleep_interval=5):
        """
        Web scrape google news rss feeds. 
        - Use date(YYYY, MM, DD) to initiate date arguments. eg: date(2025, 2, 19)
        Args:
            search_query (String): Search query
            start_time (Date): Default to datetime.now(). 
            end_time (Date): Default to datetime.now(). 
            scrape_period (int): no. of days for each scrape instance (inclusive of start and end: must be more than 1)
            add_url_parameters (dict): Additional url parameters
            save_state (boolean): Save state progress in json file. format: f"/scrape_state_{datetime.now().strftime('%Y%m%d%H%M%S')}.json"
            save_result (boolean): Save dataframe as pickle file. format: f"/scrape_result_{datetime.now().strftime('%Y%m%d%H%M%S')}.pkl"
            save_folder (String): root folder to save state and result
            preview_result (boolean): Show pandas dataframe after processing
            sleep_interval (int): Sleep for 3 seconds for every X interations (intervals).
        """
        if end_time < start_time:
            raise ValueError("End time must be greater than start time!")
        elif scrape_period < 1:
            raise ValueError("Scrape period must be more than 1!")
        
        # Save state
        self.state = {
            "parameters": {
                "search_query": search_query,
                "start_time": start_time,
                "end_time": end_time,
                "scrape_period": scrape_period,
                "add_url_parameters": add_url_parameters
            },
            "iter_no": 0
        }
        to_time = None
        df = pd.DataFrame()
        try:
            scrape_period -= 1
            while start_time <= end_time:
                to_time = min(start_time + timedelta(days=scrape_period), end_time)
                # Format and encode URL
                url_parameters = {"q": f"{search_query} after:{start_time.strftime('%Y-%m-%d')} before:{to_time.strftime('%Y-%m-%d')}", **add_url_parameters}
                url = self.base_url + urllib.parse.urlencode(url_parameters)

                self.state["to_time"] = to_time
                self.state["url"] = url
                
                # GET request and parse
                df_intermediate = self.fetch_and_parse(url, pandas= True)
                if len(df_intermediate) > 0:
                    df = pd.concat([df,df_intermediate])
                    df.drop_duplicates(subset=["pubDate", "title", "source"], inplace=True)
                    df.sort_values(by='pubDate', inplace=True)
                    df.reset_index(drop=True, inplace=True)
  
                self.state["iter_no"] += 1
                if self.state["iter_no"] % sleep_interval == 0:
                    print(f"-- Current state: {start_time}-{to_time}, record_count: {len(df)}, iter: {self.state['iter_no']}")
                    time.sleep(3)
                start_time += timedelta(days=scrape_period+1)  
        except Exception as e:
            print(f"########## Error: {start_time}-{to_time} {end_time} #########")
            print(traceback.format_exc())
        finally:
            self.state["record_count"] = len(df)
            if preview_result:
                print(f"############## Result: {len(df)} records ##############")
                display(df.tail())
            if save_state:
                save_path = save_folder.rstrip("/") + f"/scrape_state_{datetime.now().strftime('%Y%m%d%H%M%S')}.json"
                self.state["save_state_path"] = save_path
                with open(save_path, "w") as f:
                    json.dump(self.state, f, indent=4, default=str)
                print("Saved state to:", save_path)
            if save_result and len(df) > 0:
                save_path = save_folder.rstrip("/") + f"/scrape_result_{datetime.now().strftime('%Y%m%d%H%M%S')}.pkl"
                self.state["save_result_path"] = save_path
                df.to_pickle(save_path)
                print("Saved pickle to:", save_path)
            

    def fetch_and_parse(self, url, pandas= True):
        response = requests.get(url, proxies=self.proxy) 
        soup = bs.BeautifulSoup(response.text,'xml')

        result = []
        for item in soup.find_all('item'):
            result.append({
                "title": item.title.text,
                "link": item.link.text,
                "pubDate": item.pubDate.text,
                "source": item.source.text,
                "description": item.description.text,
            })
        if pandas:
            if len(result) == 0:
                return pd.DataFrame()
            df = pd.DataFrame(result)
            df['pubDate'] = pd.to_datetime(df['pubDate'], format='%a, %d %b %Y %H:%M:%S %Z', utc=True)
            return df
        return result

In [7]:
rss_feed_scraper = GoogleRSSFeedScraper()

In [74]:
banks_arr = ["UBS", "Citibank", "HSBC", "JPMorgan", "Goldman Sachs", "Credit Suisse", "Deutsche Bank", "Barclays", "Bank of America"]
for each_bank in banks_arr:
    print(f"-------------- {each_bank}  --------------")
    rss_feed_scraper.scrape(f"{each_bank} litigation",start_time=date(2000,1,1), save_state=True, save_result=True, preview_result=True, scrape_period = 30) 

-------------- UBS  --------------
-- Current state: 2000-05-30-2000-05-29, record_count: 0, iter: 5
-- Current state: 2000-10-27-2000-10-26, record_count: 1, iter: 10
-- Current state: 2001-03-26-2001-03-25, record_count: 1, iter: 15
-- Current state: 2001-08-23-2001-08-22, record_count: 1, iter: 20
-- Current state: 2002-01-20-2002-01-19, record_count: 1, iter: 25
-- Current state: 2002-06-19-2002-06-18, record_count: 1, iter: 30
-- Current state: 2002-11-16-2002-11-15, record_count: 2, iter: 35
-- Current state: 2003-04-15-2003-04-14, record_count: 2, iter: 40
-- Current state: 2003-09-12-2003-09-11, record_count: 2, iter: 45
-- Current state: 2004-02-09-2004-02-08, record_count: 3, iter: 50
-- Current state: 2004-07-08-2004-07-07, record_count: 3, iter: 55
-- Current state: 2004-12-05-2004-12-04, record_count: 4, iter: 60
-- Current state: 2005-05-04-2005-05-03, record_count: 4, iter: 65
-- Current state: 2005-10-01-2005-09-30, record_count: 4, iter: 70
-- Current state: 2006-02-28

,title,link,pubDate,source,description
617,Switzerland's Financial Sector Faces Major Scr...,https://news.google.com/rss/articles/CBMiqwFBV...,2025-02-06 17:25:51+00:00,Evrim Ağacı,"<a href=""https://news.google.com/rss/articles/..."
618,International Investors File Claims Against Sw...,https://news.google.com/rss/articles/CBMitAFBV...,2025-02-07 08:00:00+00:00,Evrim Ağacı,"<a href=""https://news.google.com/rss/articles/..."
619,UBS whistleblower verdict thrown out despite U...,https://news.google.com/rss/articles/CBMiugFBV...,2025-02-10 08:00:00+00:00,Reuters,"<a href=""https://news.google.com/rss/articles/..."
620,2nd Circ. Backs UBS In Retaliation Case That J...,https://news.google.com/rss/articles/CBMivgFBV...,2025-02-10 20:12:00+00:00,Law360,"<a href=""https://news.google.com/rss/articles/..."
621,Axsome therapeutics CFO sells $1.95 million in...,https://news.google.com/rss/articles/CBMiywFBV...,2025-02-15 03:38:31+00:00,MSN,"<a href=""https://news.google.com/rss/articles/..."


Saved state to: ./data/scrape_state_20250219095309.json
Saved pickle to: ./data/scrape_result_20250219095309.pkl
-------------- Citibank  --------------
-- Current state: 2000-05-30-2000-05-29, record_count: 0, iter: 5
-- Current state: 2000-10-27-2000-10-26, record_count: 0, iter: 10
-- Current state: 2001-03-26-2001-03-25, record_count: 0, iter: 15
-- Current state: 2001-08-23-2001-08-22, record_count: 0, iter: 20
-- Current state: 2002-01-20-2002-01-19, record_count: 0, iter: 25
-- Current state: 2002-06-19-2002-06-18, record_count: 0, iter: 30
-- Current state: 2002-11-16-2002-11-15, record_count: 0, iter: 35
-- Current state: 2003-04-15-2003-04-14, record_count: 0, iter: 40
-- Current state: 2003-09-12-2003-09-11, record_count: 0, iter: 45
-- Current state: 2004-02-09-2004-02-08, record_count: 0, iter: 50
-- Current state: 2004-07-08-2004-07-07, record_count: 0, iter: 55
-- Current state: 2004-12-05-2004-12-04, record_count: 0, iter: 60
-- Current state: 2005-05-04-2005-05-03, rec

,title,link,pubDate,source,description
510,DC Law Firms Saw Revenue and PEP Up 9% Last Ye...,https://news.google.com/rss/articles/CBMixgFBV...,2025-02-12 20:09:00+00:00,Law.com,"<a href=""https://news.google.com/rss/articles/..."
511,FEMA funding fracas in NYC - POLITICO,https://news.google.com/rss/articles/CBMiugFBV...,2025-02-12 22:42:25+00:00,POLITICO,"<a href=""https://news.google.com/rss/articles/..."
512,Trump’s Funding Freeze Raises a New Question: ...,https://news.google.com/rss/articles/CBMivAFBV...,2025-02-14 14:48:01+00:00,The New York Times,"<a href=""https://news.google.com/rss/articles/..."
513,Citibank Class Action Settlement Payout 2025: ...,https://news.google.com/rss/articles/CBMid0FVX...,2025-02-15 11:02:11+00:00,Bakhtiyarpur College of ...,"<a href=""https://news.google.com/rss/articles/..."
514,Kossoff Ch. 7 Trustee Can Pursue Clawbacks Aft...,https://news.google.com/rss/articles/CBMipgFBV...,2025-02-18 21:12:00+00:00,Law360,"<a href=""https://news.google.com/rss/articles/..."


Saved state to: ./data/scrape_state_20250219095734.json
Saved pickle to: ./data/scrape_result_20250219095734.pkl
-------------- HSBC  --------------
-- Current state: 2000-05-30-2000-05-29, record_count: 0, iter: 5
-- Current state: 2000-10-27-2000-10-26, record_count: 0, iter: 10
-- Current state: 2001-03-26-2001-03-25, record_count: 0, iter: 15
-- Current state: 2001-08-23-2001-08-22, record_count: 0, iter: 20
-- Current state: 2002-01-20-2002-01-19, record_count: 0, iter: 25
-- Current state: 2002-06-19-2002-06-18, record_count: 0, iter: 30
-- Current state: 2002-11-16-2002-11-15, record_count: 0, iter: 35
-- Current state: 2003-04-15-2003-04-14, record_count: 0, iter: 40
-- Current state: 2003-09-12-2003-09-11, record_count: 0, iter: 45
-- Current state: 2004-02-09-2004-02-08, record_count: 0, iter: 50
-- Current state: 2004-07-08-2004-07-07, record_count: 0, iter: 55
-- Current state: 2004-12-05-2004-12-04, record_count: 0, iter: 60
-- Current state: 2005-05-04-2005-05-03, record_

...

Saved state to: ./data/scrape_state_20250219102003.json
Saved pickle to: ./data/scrape_result_20250219102003.pkl
-------------- Barclays  --------------
-- Current state: 2000-05-30-2000-05-29, record_count: 0, iter: 5
-- Current state: 2000-10-27-2000-10-26, record_count: 0, iter: 10
-- Current state: 2001-03-26-2001-03-25, record_count: 1, iter: 15
-- Current state: 2001-08-23-2001-08-22, record_count: 1, iter: 20
-- Current state: 2002-01-20-2002-01-19, record_count: 1, iter: 25
-- Current state: 2002-06-19-2002-06-18, record_count: 1, iter: 30
-- Current state: 2002-11-16-2002-11-15, record_count: 1, iter: 35
-- Current state: 2003-04-15-2003-04-14, record_count: 1, iter: 40
-- Current state: 2003-09-12-2003-09-11, record_count: 1, iter: 45
-- Current state: 2004-02-09-2004-02-08, record_count: 2, iter: 50
-- Current state: 2004-07-08-2004-07-07, record_count: 2, iter: 55
-- Current state: 2004-12-05-2004-12-04, record_count: 2, iter: 60
-- Current state: 2005-05-04-2005-05-03, rec

,title,link,pubDate,source,description
852,"Barclays' Q4 Earnings Up on Solid IB, 2025 NII...",https://news.google.com/rss/articles/CBMigAFBV...,2025-02-13 12:40:00+00:00,Yahoo Finance,"<a href=""https://news.google.com/rss/articles/..."
853,Barclays: Strong Revenue Growth and Solid Outl...,https://news.google.com/rss/articles/CBMiqAFBV...,2025-02-13 13:01:46+00:00,TipRanks,"<a href=""https://news.google.com/rss/articles/..."
854,Barclays Reveals FCA Probe Over Money Launderi...,https://news.google.com/rss/articles/CBMitwFBV...,2025-02-13 19:26:00+00:00,Law360,"<a href=""https://news.google.com/rss/articles/..."
855,Barclays reserves £90m for motor finance claim...,https://news.google.com/rss/articles/CBMinAFBV...,2025-02-14 10:36:43+00:00,Law Gazette,"<a href=""https://news.google.com/rss/articles/..."
856,General counsels describe internal investigati...,https://news.google.com/rss/articles/CBMi4wFBV...,2025-02-18 12:26:30+00:00,Global Investigations Review,"<a href=""https://news.google.com/rss/articles/..."


Saved state to: ./data/scrape_state_20250219102428.json
Saved pickle to: ./data/scrape_result_20250219102428.pkl
-------------- Bank of America  --------------
-- Current state: 2000-05-30-2000-05-29, record_count: 0, iter: 5
-- Current state: 2000-10-27-2000-10-26, record_count: 0, iter: 10
-- Current state: 2001-03-26-2001-03-25, record_count: 0, iter: 15
-- Current state: 2001-08-23-2001-08-22, record_count: 1, iter: 20
-- Current state: 2002-01-20-2002-01-19, record_count: 1, iter: 25
-- Current state: 2002-06-19-2002-06-18, record_count: 1, iter: 30
-- Current state: 2002-11-16-2002-11-15, record_count: 1, iter: 35
-- Current state: 2003-04-15-2003-04-14, record_count: 1, iter: 40
-- Current state: 2003-09-12-2003-09-11, record_count: 1, iter: 45
-- Current state: 2004-02-09-2004-02-08, record_count: 1, iter: 50
-- Current state: 2004-07-08-2004-07-07, record_count: 3, iter: 55
-- Current state: 2004-12-05-2004-12-04, record_count: 3, iter: 60
-- Current state: 2005-05-04-2005-05-

,title,link,pubDate,source,description
1976,Peloton wins dismissal of shareholder lawsuit ...,https://news.google.com/rss/articles/CBMiwwFBV...,2025-02-14 23:42:57+00:00,Reuters,"<a href=""https://news.google.com/rss/articles/..."
1977,Mexican president threatens to sue Google over...,https://news.google.com/rss/articles/CBMiqwFBV...,2025-02-18 16:06:03+00:00,Global News Toronto,"<a href=""https://news.google.com/rss/articles/..."
1978,XRP ETF Gets A Shot! Why MEMX’s SEC Filing Is ...,https://news.google.com/rss/articles/CBMiX0FVX...,2025-02-18 16:30:00+00:00,TronWeekly,"<a href=""https://news.google.com/rss/articles/..."
1979,Merrill Lynch Ex-Advisers Look to Pause Deferr...,https://news.google.com/rss/articles/CBMitgFBV...,2025-02-18 18:52:00+00:00,Bloomberg Law,"<a href=""https://news.google.com/rss/articles/..."
1980,"FreightCar America, Inc. Announces New $35 Mil...",https://news.google.com/rss/articles/CBMi7gFBV...,2025-02-18 21:33:04+00:00,The Manila Times,"<a href=""https://news.google.com/rss/articles/..."


Saved state to: ./data/scrape_state_20250219102912.json
Saved pickle to: ./data/scrape_result_20250219102912.pkl


# Bloomberg law

In [ ]:
import requests
import json
import traceback
import time
import bs4 as bs
import pandas as pd
import urllib.parse
from datetime import datetime, timedelta, date
from IPython.display import display

class BloombergLawScraper():
    def __init__(self, proxy=None):
        self.proxy = proxy
        self.state = {}
        self.base_url = "https://news.bloomberglaw.com/api/v1/rss/litigation?"
    
    def scrape(self, search_query, start_time=datetime.now().date(), end_time = datetime.now().date(), fetch_limit=10, scrape_period = 7, add_url_parameters={}, save_state=False, save_result=False, save_folder="./data", preview_result=True, sleep_interval=5):
        """
        Web scrape google news rss feeds. 
        - Use date(YYYY, MM, DD) to initiate date arguments. eg: date(2025, 2, 19)
        Args:
            search_query (String): Search query
            start_time (Date): Default to datetime.now(). 
            end_time (Date): Default to datetime.now(). 
            fetch_limit (int): Number of items fetch (default is 10)
            scrape_period (int): no. of days for each scrape instance (inclusive of start and exclusive end: must be more than 1)
            add_url_parameters (dict): Additional url parameters
            save_state (boolean): Save state progress in json file. format: f"/scrape_state_{datetime.now().strftime('%Y%m%d%H%M%S')}.json"
            save_result (boolean): Save dataframe as pickle file. format: f"/scrape_result_{datetime.now().strftime('%Y%m%d%H%M%S')}.pkl"
            save_folder (String): root folder to save state and result
            preview_result (boolean): Show pandas dataframe after processing
            sleep_interval (int): Sleep for 3 seconds for every X interations (intervals).
        """
        if end_time < start_time:
            raise ValueError("End time must be greater than start time!")
        elif scrape_period < 1:
            raise ValueError("Scrape period must be more than 1!")
        
        # Save state
        self.state = {
            "parameters": {
                "search_query": search_query,
                "start_time": start_time,
                "end_time": end_time,
                "scrape_period": scrape_period,
                "add_url_parameters": add_url_parameters,
                "fetch_limit": fetch_limit
            },
            "iter_no": 0
        }
        to_time = None
        df = pd.DataFrame()
        try:
            while start_time <= end_time:
                to_time = min(start_time + timedelta(days=scrape_period), end_time)
                # Format and encode URL
                url_parameters = {"query": search_query, "startDate": start_time.strftime('%Y-%m-%d'),"endDate": to_time.strftime('%Y-%m-%d'), "limit": fetch_limit, **add_url_parameters}
                url = self.base_url + urllib.parse.urlencode(url_parameters)

                self.state["to_time"] = to_time
                self.state["url"] = url
                
                # GET request and parse
                df_intermediate = self.fetch_and_parse(url, pandas= True)
                if len(df_intermediate) > 0:
                    df = pd.concat([df,df_intermediate])
                    df.drop_duplicates(subset=["pubDate", "title"], inplace=True)
                    df.sort_values(by='pubDate', inplace=True)
                    df.reset_index(drop=True, inplace=True)

                self.state["iter_no"] += 1
                if self.state["iter_no"] % sleep_interval == 0:
                    print(f"-- Current state: {start_time}-{to_time}, record_count: {len(df)}, iter: {self.state['iter_no']}")
                    time.sleep(3)
                start_time += timedelta(days=scrape_period) 
        except Exception as e:
            print(f"########## Error: {start_time}-{to_time} {end_time} #########")
            print(traceback.format_exc())
        finally:
            self.state["record_count"] = len(df)
            if preview_result:
                print(f"############## Result: {len(df)} records ##############")
                display(df.tail())
            if save_state:
                save_path = save_folder.rstrip("/") + f"/scrape_state_{datetime.now().strftime('%Y%m%d%H%M%S')}.json"
                self.state["save_state_path"] = save_path
                with open(save_path, "w") as f:
                    json.dump(self.state, f, indent=4, default=str)
                print("Saved state to:", save_path)
            if save_result and len(df) > 0:
                save_path = save_folder.rstrip("/") + f"/scrape_result_{datetime.now().strftime('%Y%m%d%H%M%S')}.pkl"
                self.state["save_result_path"] = save_path
                df.to_pickle(save_path)
                print("Saved pickle to:", save_path)
            

    def fetch_and_parse(self, url, pandas= True):
        response = requests.get(url, proxies=self.proxy) 
        soup = bs.BeautifulSoup(response.text,'xml')

        result = []
        for item in soup.find_all('item'):
            result.append({
                "title": item.title.text,
                "link": item.link.text,
                "pubDate": item.pubDate.text,
                "description": item.description.text,
                "topic": list(map(lambda x: x.text, item.find_all('md:topic')))
            })
        if pandas:
            if len(result) == 0:
                return pd.DataFrame()
            df = pd.DataFrame(result)
            df['pubDate'] = pd.to_datetime(df['pubDate'], format='%a, %d %b %Y %H:%M:%S %z', utc=True)
            return df
        return result

In [27]:
bloombergLawScraper = BloombergLawScraper()

In [28]:
banks_arr = ["UBS", "Citibank", "HSBC", "JPMorgan", "Goldman Sachs", "Credit Suisse", "Deutsche Bank", "Barclays", "Bank of America"]
for each_bank in banks_arr:
    print(f"-------------- {each_bank}  --------------")
    bloombergLawScraper.scrape(each_bank, start_time=date(2000,1,1), save_state=True, save_result=True, preview_result=True, scrape_period = 30, fetch_limit=100) 

-------------- UBS  --------------
-- Current state: 2000-04-30-2000-05-30, record_count: 0, iter: 5
-- Current state: 2000-09-27-2000-10-27, record_count: 0, iter: 10
-- Current state: 2001-02-24-2001-03-26, record_count: 0, iter: 15
-- Current state: 2001-07-24-2001-08-23, record_count: 0, iter: 20
-- Current state: 2001-12-21-2002-01-20, record_count: 0, iter: 25
-- Current state: 2002-05-20-2002-06-19, record_count: 0, iter: 30
-- Current state: 2002-10-17-2002-11-16, record_count: 0, iter: 35
-- Current state: 2003-03-16-2003-04-15, record_count: 0, iter: 40
-- Current state: 2003-08-13-2003-09-12, record_count: 0, iter: 45
-- Current state: 2004-01-10-2004-02-09, record_count: 0, iter: 50
-- Current state: 2004-06-08-2004-07-08, record_count: 0, iter: 55
-- Current state: 2004-11-05-2004-12-05, record_count: 0, iter: 60
-- Current state: 2005-04-04-2005-05-04, record_count: 0, iter: 65
-- Current state: 2005-09-01-2005-10-01, record_count: 0, iter: 70
-- Current state: 2006-01-29

C:\Users\gansh\AppData\Local\Temp\ipykernel_25860\407789001.py:114: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['pubDate'] = pd.to_datetime(df['pubDate'], format='%a, %d %b %Y %H:%M:%S %z')


-- Current state: 2023-04-30-2023-05-30, record_count: 59, iter: 285
-- Current state: 2023-09-27-2023-10-27, record_count: 89, iter: 290


C:\Users\gansh\AppData\Local\Temp\ipykernel_25860\407789001.py:114: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['pubDate'] = pd.to_datetime(df['pubDate'], format='%a, %d %b %Y %H:%M:%S %z')
C:\Users\gansh\AppData\Local\Temp\ipykernel_25860\407789001.py:114: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['pubDate'] = pd.to_datetime(df['pubDate'], format='%a, %d %b %Y %H:%M:%S %z')


-- Current state: 2024-02-24-2024-03-25, record_count: 106, iter: 295
-- Current state: 2024-07-23-2024-08-22, record_count: 158, iter: 300


...

-- Current state: 2024-02-24-2024-03-25, record_count: 771, iter: 295
-- Current state: 2024-07-23-2024-08-22, record_count: 1021, iter: 300


C:\Users\gansh\AppData\Local\Temp\ipykernel_25860\407789001.py:114: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['pubDate'] = pd.to_datetime(df['pubDate'], format='%a, %d %b %Y %H:%M:%S %z')


-- Current state: 2024-12-20-2025-01-19, record_count: 1193, iter: 305
############## Result: 1242 records ##############


,title,link,pubDate,description,topic
1237,Seven Cases to Watch as the Courts Weigh Trump...,https://news.bloomberglaw.com/litigation/seven...,2025-02-15 08:00:22-05:00,President Donald Trump is now defending his wa...,"[gender identity discrimination, unfair and de..."
1238,Willkie Removal in Bankruptcy Case Is Lesson i...,https://news.bloomberglaw.com/litigation/willk...,2025-02-18 05:00:01-05:00,Willkie Farr & Gallagher LLP's disqualificatio...,"[attorney disqualification, leveraged buyout, ..."
1239,Black Church Breaks New Legal Ground Taking Ov...,https://news.bloomberglaw.com/litigation/black...,2025-02-18 05:05:02-05:00,A historic Black church unable to compel the f...,"[default judgment, goodwill, freedom of speech..."
1240,What J&J Is Trying to Achieve in Bankruptcy Co...,https://news.bloomberglaw.com/litigation/what-...,2025-02-18 10:00:03-05:00,Johnson & Johnson begins a two-week trial on F...,"[settlements, personal use product safety, pha..."
1241,Merrill Lynch Ex-Advisers Look to Pause Deferr...,https://news.bloomberglaw.com/litigation/merri...,2025-02-18 13:52:33-05:00,Seven former financial advisers for Bank of Am...,"[forfeiture, commission payments, mandatory ar..."


Saved state to: ./data/scrape_state_20250219132236.json
Saved pickle to: ./data/scrape_result_20250219132236.pkl
